In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
data1=pd.read_csv("/kaggle/input/movies-dataset-for-feature-extracion-prediction/movies.csv")
data1.head()

# **FEATURE ENGINEERING**

In [ ]:
data1.describe(include="all")

In [ ]:
data1.info

In [ ]:
data1.isna().sum()

In [ ]:
remove=[]
for i in data1.columns:
    if data1[i].isnull().sum()>len(data1)/2:
        remove.append(i)
    elif data1[i].isnull().sum()!=0:
        data1[i].fillna(method="ffill",inplace=True)
data1.drop(remove,axis=1,inplace=True)

In [ ]:
data1["YEAR"]=data1["YEAR"].str.replace("\D","")
data1["YEAR"]=data1["YEAR"].map(lambda x:list(str(x)[:4])).map(lambda x:"".join(x))

In [ ]:
data1["STARS"]=data1["STARS"].str.replace("\n","")

In [ ]:
data1["GENRE"]=data1["GENRE"].apply(lambda x:str(x).strip("\n"))

In [ ]:
data1["ONE-LINE"]=data1["ONE-LINE"].apply(lambda x:str(x).strip("\n"))

# BUILD RECOMMENDATION SYSTEM

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')

In [ ]:
tfidf_matrix = tfidf.fit_transform(data1["ONE-LINE"])

In [ ]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
indices=pd.Series(data1.index, index=data1['MOVIES']).drop_duplicates()

In [ ]:
def get_recommendations(title, cosine_sim=cosine_sim):
    
    idx = indices[title]

    sim_scores = list(enumerate(cosine_sim[idx]))

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:11]

    movie_indices = [i[0] for i in sim_scores]

    return data1['MOVIES'].iloc[movie_indices]

In [ ]:
get_input="Sweet Tooth"  #give (Sweet Tooth,Tidelands)
get_recommendations(get_input)